# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 
%env


{'ACLOCAL_PATH': 'C:\\Program Files\\Git\\mingw64\\share\\aclocal;C:\\Program Files\\Git\\usr\\share\\aclocal',
 'ALLUSERSPROFILE': 'C:\\ProgramData',
 'APPDATA': 'C:\\Users\\hyunh\\AppData\\Roaming',
 'CHROME_CRASHPAD_PIPE_NAME': '\\\\.\\pipe\\crashpad_15832_PSRFJSKEAPAJOMHO',
 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
 'COMPUTERNAME': 'HUHU-SURFACE',
 'COMSPEC': 'C:\\windows\\system32\\cmd.exe',
 'CONDA_DEFAULT_ENV': 'dsi_participant',
 'CONDA_EXE': 'C:\\Users\\hyunh\\AppData\\Local\\miniconda\\condabin\\..\\Scripts\\conda.exe',
 'CONDA_EXES': '"C:\\Users\\hyunh\\AppData\\Local\\miniconda\\condabin\\..\\Scripts\\conda.exe"  ',
 'CONDA_PREFIX': 'C:\\Users\\hyunh\\AppData\\Local\\miniconda\\envs\\dsi_participant',
 'CONDA_PROMPT_MODIFIER': '(dsi_participant) ',
 'CONDA_PYTHON_EXE': 'C:/Users/hyunh/AppData/Local/miniconda/python.exe',
 'CONDA_ROOT

In [2]:
import dask.dataframe as dd

c:\Users\hyunh\AppData\Local\miniconda\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
C:\Users\hyunh\AppData\Local\Temp\ipykernel_14836\676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any fe

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")
print(dd_px.head())

                            Date  Adj Close     Close      High       Low  \
Ticker                                                                      
AAPL   2000-01-03 00:00:00+00:00   0.843077  0.999442  1.004464  0.907924   
AAPL   2000-01-04 00:00:00+00:00   0.771996  0.915179  0.987723  0.903460   
AAPL   2000-01-05 00:00:00+00:00   0.783293  0.928571  0.987165  0.919643   
AAPL   2000-01-06 00:00:00+00:00   0.715509  0.848214  0.955357  0.848214   
AAPL   2000-01-07 00:00:00+00:00   0.749401  0.888393  0.901786  0.852679   

            Open       Volume  Year  
Ticker                               
AAPL    0.936384  535796800.0  2000  
AAPL    0.966518  512377600.0  2000  
AAPL    0.926339  778321600.0  2000  
AAPL    0.947545  767972800.0  2000  
AAPL    0.861607  460734400.0  2000  


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
# Write your code below.
dd_feat = dd_px.groupby("Ticker", group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1=x["Close"].shift(1),
        Adj_Close_lag_1=x["Adj Close"].shift(1),
        returns = (x["Close"] / x["Close"].shift(1)) - 1,
        hi_lo_range = x["High"] - x["Low"]
    )
)


C:\Users\hyunh\AppData\Local\Temp\ipykernel_14836\942345666.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby("Ticker", group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [15]:
# Write your code below.
df_feat = dd_feat.compute().reset_index().head()

In [18]:
print(df_feat.columns)
df_feat = df_feat.sort_values(["Ticker", "Date"])
df_feat["returns_ma_10"] = df_feat.groupby("Ticker")["returns"].rolling(10).mean().reset_index(level=0, drop=True)
print(df_feat.head())

Index(['Ticker', 'Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume',
       'Year', 'Close_lag_1', 'Adj_Close_lag_1', 'returns', 'hi_lo_range'],
      dtype='object')
  Ticker                      Date  Adj Close     Close      High       Low  \
0   AAPL 2000-01-03 00:00:00+00:00   0.843077  0.999442  1.004464  0.907924   
1   AAPL 2000-01-04 00:00:00+00:00   0.771996  0.915179  0.987723  0.903460   
2   AAPL 2000-01-05 00:00:00+00:00   0.783293  0.928571  0.987165  0.919643   
3   AAPL 2000-01-06 00:00:00+00:00   0.715509  0.848214  0.955357  0.848214   
4   AAPL 2000-01-07 00:00:00+00:00   0.749401  0.888393  0.901786  0.852679   

       Open       Volume  Year  Close_lag_1  Adj_Close_lag_1   returns  \
0  0.936384  535796800.0  2000          NaN              NaN       NaN   
1  0.966518  512377600.0  2000     0.999442         0.843077 -0.084310   
2  0.926339  778321600.0  2000     0.915179         0.771996  0.014633   
3  0.947545  767972800.0  2000     0.928571        

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
  + I think it was not necessary to convert pandas to perform the calculation, however, to be able to simplify the calculation in this case (the dataset is not too huge). We may achieve similar result using Dask transformation ".apply"
 

+ Would it have been better to do it in Dask? Why?
  + If the dataset is huge and consume extensive time to be processed in pandas, then computing in Dask is ideal as it can process data both lazily and using distributed computation.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.